In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.transforms import ToTensor
from pandas import Series
import numpy as np
from sklearn import preprocessing

In [ ]:
#The dataset, collected on the Marconi 100 system, has been published and
#described in the Scientific dataset publication: https://www.nature.com/articles/s41597-023-02174-3
#It is available on Zenodo: https://zenodo.org/records/7541722


#This is rack 1 of the dataset
!wget https://zenodo.org/record/7541722/files/1.tar?download=1 -o rack.tar

In [ ]:
!tar -xf /content/1.tar?download=1

In [ ]:
#This is node 36, of rack 1
file_name = '36.parquet'

In [ ]:
DATA = pd.read_parquet(file_name)
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,22.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399998,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,20.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>


In [ ]:
#The dataset has periods of unavailability of Nagios traces, as described in the dataset paper:  https://www.nature.com/articles/s41597-023-02174-3/figures/3
#We suggest dropping the periods where the labels are largely unavailable and using the dataset either after 1.4.2021 or 1.10.2021.
DATA = DATA[DATA['timestamp'] > '2021-04-01']

### parisa change
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)
## parisa change

<ipython-input-6-07ca0595a954>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['value'] = DATA['value'].replace(2,1)


In [ ]:
DATA.reset_index(drop=True, inplace = True)
DATA = DATA.fillna(0)
DATA['timestamp'] = pd.to_datetime(DATA['timestamp'])
l = DATA.timestamp.diff() == pd.Timedelta(minutes=15) #time consistency -> measurements should be 15 minutes apart
chunks = []
current_chunk = []
for index, value in enumerate(l):
    current_chunk.append(DATA.iloc[index])
    if not value:
        chunks.append(pd.DataFrame(current_chunk))
        current_chunk = []


if current_chunk:
    chunks.append(pd.DataFrame(current_chunk))

In [ ]:
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change
relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
revelent_len = [len(c) for c in chunks if len(c) >= 20]
DATA2 = pd.concat(relevant)
DATA = DATA2
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change end
##3 parisa change
# relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
# DATA2 = pd.concat(relevant)
# DATA = DATA2
# DATA['value'] = DATA['value'].replace(2,1)
# DATA['value'] = DATA['value'].replace(3,1)
# DATA = DATA.drop(columns=['timestamp'])
# DATA = DATA.astype(float)
##3 parisa change end

In [ ]:
DATA[DATA["value"]>0]

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
102,2021-04-03 02:00:00+00:00,24.822222,0.075686,24.600000,25.000000,33.000000,0.000000,33.0,33.0,33.244444,...,64.0,12.200001,0.000000,12.2,12.2,688.000000,290.513721,520.0,1560.0,1
103,2021-04-03 02:15:00+00:00,24.955556,0.083148,24.799999,25.000000,33.000000,0.000000,33.0,33.0,33.777778,...,73.0,12.200001,0.000000,12.2,12.2,687.111111,276.972700,520.0,1560.0,1
104,2021-04-03 02:30:00+00:00,24.951112,0.085952,24.799999,25.000000,33.000000,0.000000,33.0,33.0,33.800000,...,77.0,12.200001,0.000000,12.2,12.2,683.111111,286.122136,500.0,1520.0,1
107,2021-04-03 03:15:00+00:00,24.800003,0.000000,24.799999,24.799999,33.000000,0.000000,33.0,33.0,33.466667,...,65.0,12.200001,0.000000,12.2,12.2,599.111111,180.071865,520.0,1440.0,1
108,2021-04-03 03:30:00+00:00,24.737779,0.092589,24.600000,24.799999,33.000000,0.000000,33.0,33.0,33.088889,...,59.0,12.200001,0.000000,12.2,12.2,691.111111,270.096462,520.0,1420.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44702,2022-07-11 16:00:00+00:00,22.835555,0.076465,22.799999,23.000000,32.355556,1.267641,31.0,34.0,33.755556,...,73.0,12.200001,0.000000,12.2,12.2,704.000000,186.313952,540.0,1520.0,1
45628,2022-07-21 07:30:00+00:00,22.808889,0.041216,22.799999,23.000000,31.000000,0.000000,31.0,31.0,31.000000,...,61.0,12.206668,0.024944,12.2,12.3,687.111111,217.466444,520.0,1640.0,1
46793,2022-08-02 10:45:00+00:00,22.226670,0.067987,22.200001,22.400000,29.000000,0.000000,29.0,29.0,30.000000,...,31.0,12.364446,0.076465,12.2,12.4,454.666667,99.098156,400.0,700.0,1
46794,2022-08-02 11:00:00+00:00,22.217780,0.056916,22.200001,22.400000,29.000000,0.000000,29.0,29.0,30.000000,...,21.0,12.400001,0.000000,12.4,12.4,405.777778,9.064924,400.0,420.0,1


In [ ]:
DATA3= DATA
DATA=DATA.drop(columns=['timestamp'])
original_dtypes = DATA.dtypes
DATA = DATA.astype(float)

In [ ]:
#### add label of next row
DATA['next_value'] = DATA['value'].shift(-1).fillna(0)

In [ ]:
# df_value_0 = DATA[DATA['value'] == 0.0].drop(columns=['value']).to_numpy()

# # Separate rows where 'value' is 1.0
# df_value_1 = DATA[DATA['value'] == 1.0].drop(columns=['value']).to_numpy()

In [ ]:
### use for delete value column
# DATA=DATA.drop(columns=['value'])
original_dtypes = DATA.dtypes

In [ ]:
### use for delete value column
# scaler = preprocessing.MinMaxScaler()  ### use further
# names = DATA.columns  ##
# d_0 = scaler.fit_transform(df_value_0)
# d_1= scaler.fit_transform(df_value_1)
# df_scale_1 = pd.DataFrame(d_1, columns=names)
# df_scale_0 = pd.DataFrame(d_0, columns=names)

In [ ]:
scaler = preprocessing.MinMaxScaler()  ### use further
names = DATA.columns  ##
d = scaler.fit_transform(DATA)
DATA = pd.DataFrame(d, columns=names)
# ### parisa change                                --- here to change more or add time stamp in train

#timestamp_scaler = preprocessing.MinMaxScaler()
#DATA['timestamp'] = timestamp_scaler.fit_transform(DATA3['timestamp'].astype('int64').values.reshape(-1, 1))
#### parisa change end

In [ ]:
# train_data = DATA[:int(DATA.shape[0]*0.8)]
# test_data = DATA[int(DATA.shape[0]*0.8):]

################# parisa change
train_data = DATA[:]

############### end parisa change

In [ ]:
df_value_1.shape

(1311, 352)

In [ ]:
df_value_0.shape

(50998, 352)

In [ ]:
train_data['value'].value_counts()

,count
value,
0.0,50998
1.0,1311


In [ ]:
# df_value_0 = df_value_0.drop(columns=['value']).to_numpy()
# # train_feat = train_feat.to_numpy()
# df_value_1 = df_value_1.drop(columns=['value']).to_numpy()
# # test_feat = test_feat.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [ ]:
train_data_tensor = torch.tensor(train_data.to_numpy(), dtype=torch.float)

In [ ]:
# test_labels = test_data['value']
# test_labels = test_labels.to_numpy()
# test_labels = torch.tensor(test_labels, dtype=torch.float)
# test_labels

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_window = 20 ## 20

def create_data_seq(data,tw):
    seq = []
    for i in range(len(data)-tw):
        x_seq = data[i:i+tw],
        y_seq = data[i+tw:i+tw+1]  ###در ترین میاد 0 تا 20 رو میده و میگه 21 رو حدس بزن
        seq.append((x_seq,y_seq))
    return seq


In [ ]:
train_seq = create_data_seq(train_data_tensor,train_window)

In [ ]:
X[0].size()


torch.Size([1, 20, 354])

In [ ]:
train_loader = DataLoader(train_seq, batch_size=1, shuffle=True)

X,y  = next(iter(train_loader))
# print(X)


In [ ]:
class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=354,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=8,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x[:, :, :]

In [ ]:
############### mine
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=353,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

        # Initial hidden and cell states for lstm2
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # Initial hidden and cell states for lstm3
        h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))

        return x[:, :, :]

In [ ]:
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=352,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x

In [ ]:
### mine
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=353,
            batch_first=True,
            num_layers=1
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

        # Initial hidden and cell states for lstm2
        h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # Initial hidden and cell states for lstm3
        h0 = torch.zeros(1, batch_size, 353).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 353).requires_grad_().to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))

        return x

In [ ]:
############### mine for add h,c
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=354,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )
        self.h_proj_1_to_2 = torch.nn.Linear(32, 16)
        self.c_proj_1_to_2 = torch.nn.Linear(32, 16)

        self.h_proj_2_to_3 = torch.nn.Linear(16, 8)
        self.c_proj_2_to_3 = torch.nn.Linear(16, 8)

    def forward(self, x):
        batch_size = x.shape[0]
        # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))

        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).to(x.device)
        c0 = torch.zeros(1, batch_size, 32).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h0, c0))

        # Resize h1, c1 to match the input size of lstm2
        h1 = self.h_proj_1_to_2(h1)
        c1 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h1, c1))

        # Resize h2, c2 to match the input size of lstm3
        h2 = self.h_proj_2_to_3(h2)
        c2 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h2, c2))
        return x[:, :, :]

In [ ]:
### mine for add h,c
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=354,
            batch_first=True,
            num_layers=1
        )

        self.h_proj_1_to_2 = torch.nn.Linear(16, 32)
        self.c_proj_1_to_2 = torch.nn.Linear(16, 32)

        self.h_proj_2_to_3 = torch.nn.Linear(32, 354)
        self.c_proj_2_to_3 = torch.nn.Linear(32, 354)

    def forward(self, x):
        batch_size = x.shape[0]

        # # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))


        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        h1 = torch.zeros(1, batch_size, 16).to(x.device)
        c1 = torch.zeros(1, batch_size, 16).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h1, c1))

        # Resize h1, c1 to match the input size of lstm2
        h2 = self.h_proj_1_to_2(h1)
        c2 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h2, c2))

        # Resize h2, c2 to match the input size of lstm3
        h3 = self.h_proj_2_to_3(h2)
        c3 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h3, c3))

        return x

In [ ]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.decoder = decoder().to(device)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8,16)
        self.fc2 = torch.nn.Linear(16,354)


    def forward(self, x):
        x = self.encoder(x)
        #print('LSTM out', x.shape)
        x = self.fc1(x)
        #print('DENSE 1', x.shape)
        x = self.fc2(x)
        #print('DENSE 2', x.shape)

        return x

In [ ]:
### mine
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8, 16)
        self.fc2 = torch.nn.Linear(16, 8)
        self.decoder = decoder().to(device)  # Adding the decoder to RUAD

    def forward(self, x):
        x = self.encoder(x)
        # Passing through the fully connected layers   ----- new changes
        # x = self.fc1(x)
        # x = self.fc2(x)
        # Passing the result through the decoder ------- end of new changes
        x = self.decoder(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = RUAD()
model

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(354, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 354, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=354, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=354,

In [ ]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)  ### adam ro bezar va bebin
criterion = torch.nn.MSELoss()

In [ ]:
for d in train_loader_value_0:
    print(len(d[0]))
    print(d[0][0].size())
    print(d[1].size())
    break
    ## batch size,(row count,column count) --> sequence

NameError: name 'train_loader_value_0' is not defined

In [ ]:
print(model)

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(353, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 353, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=353, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=353,

In [ ]:
%%time
def train(output,data):
    model.train()
    for d in data:
        #print(type(d))
        x = d[0][0].to(device)
        y = d[1].to(device)
        optimizer.zero_grad()
        out = model(x)
        # print(out[0][9])
        output.append([out[0][train_window-1]])
        loss = criterion(out[0][train_window-1], y[0][0])
        loss.backward()
        optimizer.step()
    return float(loss) , output







############ results
########### result change layer 8 to 32
# Epoch: 01, Loss: 0.0188
# Epoch: 02, Loss: 0.0273
# Epoch: 03, Loss: 0.0033
# CPU times: user 6min 3s, sys: 3.62 s, total: 6min 6s
# Wall time: 6min 12s

########### result add hidden ayer 32
# Epoch: 01, Loss: 0.0021
# Epoch: 02, Loss: 0.0378
# Epoch: 03, Loss: 0.0069
# CPU times: user 7min 58s, sys: 4.28 s, total: 8min 2s
# Wall time: 8min 11s

########### result add hidden layer 32 to decoder too and add decoder to ruad model with gpu
# Epoch: 01, Loss: 0.0200
# Epoch: 02, Loss: 0.0210
# Epoch: 03, Loss: 0.0231
# CPU times: user 13min 7s, sys: 14.9 s, total: 13min 22s
# Wall time: 13min 5s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu
# Epoch: 01, Loss: 0.0089
# Epoch: 02, Loss: 0.0483
# Epoch: 03, Loss: 0.0035
# CPU times: user 12min 38s, sys: 13.5 s, total: 12min 51s
# Wall time: 12min 34s

########### result first ruad model with gpu and add timestamp column and window 10 and seq
# Epoch: 01, Loss: 0.0107
# Epoch: 02, Loss: 0.0098
# Epoch: 03, Loss: 0.0219
# CPU times: user 11min 17s, sys: 4.6 s, total: 11min 22s
# Wall time: 11min 28s

########### result first ruad model with gpu and add timestamp column and window 10 and remove seq
# Epoch: 01, Loss: 0.0252
# Epoch: 02, Loss: 0.0093
# Epoch: 03, Loss: 0.0188
# CPU times: user 11min 8s, sys: 4.67 s, total: 11min 12s
# Wall time: 11min 19s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , adam optimizer
# Epoch: 01, Loss: 0.0365
# Epoch: 02, Loss: 0.0473
# Epoch: 03, Loss: 0.0536
# CPU times: user 12min 14s, sys: 13.8 s, total: 12min 28s
# Wall time: 12min 12s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer
# Epoch: 01, Loss: 0.0594
# Epoch: 02, Loss: 0.0260
# Epoch: 03, Loss: 0.0311
# CPU times: user 11min 27s, sys: 13.3 s, total: 11min 40s
# Wall time: 11min 24s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 8
# Epoch: 01, Loss: 0.0270
# Epoch: 02, Loss: 0.0267
# Epoch: 03, Loss: 0.0317
# Epoch: 04, Loss: 0.0313
# Epoch: 05, Loss: 0.0615
# CPU times: user 19min 21s, sys: 23.7 s, total: 19min 44s
# Wall time: 19min 18s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 5 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# CPU times: user 20min 12s, sys: 12 s, total: 20min 24s
# Wall time: 20min 39s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# Epoch: 06, Loss: 0.0030
# CPU times: user 24min 16s, sys: 14.37 s, total: 24min 30s
# Wall time: 24min 48s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0042
# Epoch: 02, Loss: 0.0035
# Epoch: 03, Loss: 0.0061
# Epoch: 04, Loss: 0.0053
# Epoch: 05, Loss: 0.0214
# Epoch: 06, Loss: 0.0109
# CPU times: user 26min 13s, sys: 14.2 s, total: 26min 27s
# Wall time: 26min 46s


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [ ]:
### before run this you need to rerun the model first
output=[]
i=0
for epoch in range(6):
    loss ,output = train(output,train_loader)
    if i!=5:
      output=[]
    i+=1
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')

# Epoch: 01, Loss: 0.0326
# Epoch: 02, Loss: 0.0253
# Epoch: 03, Loss: 0.0197
# Epoch: 04, Loss: 0.0029
# Epoch: 05, Loss: 0.0104
# Epoch: 06, Loss: 0.0124

Epoch: 01, Loss: 0.0239
Epoch: 02, Loss: 0.0223
Epoch: 03, Loss: 0.0072
Epoch: 04, Loss: 0.0094
Epoch: 05, Loss: 0.0039
Epoch: 06, Loss: 0.0034


In [ ]:
# def tensor_to_dataframe(tensor, columns, device):
#     numpy_array = tensor.cpu().numpy()  # Convert tensor to NumPy array
#     df = pd.DataFrame(numpy_array, columns=columns)  # Create DataFrame with original column names
#     return df

In [ ]:
### scale ,name
df_list=[]
for l in output:
  ### tensore darim
  numpy_array = l[0].cpu().detach().numpy().reshape(1, -1)
  df=scaler.inverse_transform(numpy_array)
  df_list.append(df)


In [ ]:
df_combined = pd.concat([pd.DataFrame(df, columns=names) for df in df_list], ignore_index=True)
df_combined=df_combined.astype(original_dtypes)

In [ ]:
df_combined.astype(original_dtypes)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,next_value
0,23.639475,0.087068,23.478762,23.145287,32.286564,0.189415,32.029263,32.344254,32.658520,0.127147,...,12.267771,-0.002707,12.269944,12.283220,727.833801,107.324013,594.849792,1005.179077,0.018354,0.005504
1,23.155619,0.079264,23.029579,23.492922,30.849302,0.158692,30.734596,31.131956,31.505634,0.128578,...,12.141682,0.004152,12.129576,12.158064,685.704834,90.303688,533.743530,934.141296,0.036854,0.041747
2,23.620388,0.090944,23.483377,23.161476,32.286709,0.190391,32.027122,32.339973,32.663158,0.127238,...,12.267597,-0.003941,12.269765,12.283132,729.221985,105.108253,595.503601,1005.394592,0.018131,0.005442
3,23.443068,0.086445,23.314333,23.486629,31.266729,0.165255,31.317989,31.625969,31.918673,0.125373,...,12.192164,-0.001451,12.189639,12.209301,702.924255,109.287445,557.373718,1009.428406,0.031648,0.030398
4,23.524555,0.095636,23.399746,23.413658,31.459009,0.168253,31.539965,31.793932,32.072010,0.124277,...,12.212283,-0.002509,12.211745,12.229104,709.356079,117.045425,566.936035,1023.458862,0.029246,0.024311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52284,23.338358,0.087100,23.064806,23.651115,30.997826,0.150974,30.907887,31.332464,31.612547,0.138143,...,12.186502,0.027379,12.177468,12.201460,676.276489,95.622787,529.458374,967.762390,0.041690,0.050543
52285,23.571321,0.103861,23.395624,23.111681,31.844189,0.187999,31.748096,32.082684,32.248878,0.132488,...,12.264905,0.012120,12.265039,12.277589,782.034058,158.724503,580.816345,1138.090576,0.017884,0.015648
52286,23.570511,0.106452,23.397423,23.101120,31.793592,0.196812,31.732630,32.081200,32.218983,0.133364,...,12.262326,0.013934,12.262407,12.274898,782.505066,167.409149,579.077026,1143.852051,0.018156,0.016489
52287,23.455093,0.097552,23.205334,23.542545,31.181665,0.166220,31.171974,31.660873,31.812592,0.136324,...,12.206349,0.025453,12.201448,12.220190,734.606079,127.388557,542.846130,1097.645996,0.033546,0.041259


In [ ]:
df_combined[df_combined["ambient_avg"]< 0.30]

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,next_value


In [ ]:
### add time after ass anomaly for not being complicated on timestamp

timestamps = pd.date_range(start=pd.Timestamp('2021-04-02 00:45:00', tz='UTC'), periods=len(df_combined), freq='15T')

# Add timestamp and value columns to the DataFrame
df_combined['timestamp'] = timestamps

<ipython-input-32-18a21b60e666>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined['timestamp'] = timestamps


In [ ]:
df_combined

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,next_value,timestamp
0,23.639475,0.087068,23.478762,23.145287,32.286564,0.189415,32.029263,32.344254,32.658520,0.127147,...,-0.002707,12.269944,12.283220,727.833801,107.324013,594.849792,1005.179077,0.018354,0.005504,2021-04-02 00:45:00+00:00
1,23.155619,0.079264,23.029579,23.492922,30.849302,0.158692,30.734596,31.131956,31.505634,0.128578,...,0.004152,12.129576,12.158064,685.704834,90.303688,533.743530,934.141296,0.036854,0.041747,2021-04-02 01:00:00+00:00
2,23.620388,0.090944,23.483377,23.161476,32.286709,0.190391,32.027122,32.339973,32.663158,0.127238,...,-0.003941,12.269765,12.283132,729.221985,105.108253,595.503601,1005.394592,0.018131,0.005442,2021-04-02 01:15:00+00:00
3,23.443068,0.086445,23.314333,23.486629,31.266729,0.165255,31.317989,31.625969,31.918673,0.125373,...,-0.001451,12.189639,12.209301,702.924255,109.287445,557.373718,1009.428406,0.031648,0.030398,2021-04-02 01:30:00+00:00
4,23.524555,0.095636,23.399746,23.413658,31.459009,0.168253,31.539965,31.793932,32.072010,0.124277,...,-0.002509,12.211745,12.229104,709.356079,117.045425,566.936035,1023.458862,0.029246,0.024311,2021-04-02 01:45:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52284,23.338358,0.087100,23.064806,23.651115,30.997826,0.150974,30.907887,31.332464,31.612547,0.138143,...,0.027379,12.177468,12.201460,676.276489,95.622787,529.458374,967.762390,0.041690,0.050543,2022-09-28 15:45:00+00:00
52285,23.571321,0.103861,23.395624,23.111681,31.844189,0.187999,31.748096,32.082684,32.248878,0.132488,...,0.012120,12.265039,12.277589,782.034058,158.724503,580.816345,1138.090576,0.017884,0.015648,2022-09-28 16:00:00+00:00
52286,23.570511,0.106452,23.397423,23.101120,31.793592,0.196812,31.732630,32.081200,32.218983,0.133364,...,0.013934,12.262407,12.274898,782.505066,167.409149,579.077026,1143.852051,0.018156,0.016489,2022-09-28 16:15:00+00:00
52287,23.455093,0.097552,23.205334,23.542545,31.181665,0.166220,31.171974,31.660873,31.812592,0.136324,...,0.025453,12.201448,12.220190,734.606079,127.388557,542.846130,1097.645996,0.033546,0.041259,2022-09-28 16:30:00+00:00


In [ ]:
df_combined[df_combined["value"]>0.1]

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,next_value,timestamp
5380,3.674841,0.159237,2.021647,3.032097,-0.703131,0.055388,-0.091342,-0.223871,0.032362,-0.025096,...,0.060087,0.165401,0.108973,1.201084,17.563082,26.262026,-71.111275,0.100315,0.088480,2021-05-28 01:45:00+00:00
5387,3.687580,0.159080,2.032548,3.045161,-0.685830,0.055430,-0.077762,-0.207337,0.052837,-0.024782,...,0.060051,0.169934,0.112766,1.524614,17.583662,26.673819,-70.931427,0.100178,0.088420,2021-05-28 03:30:00+00:00
5389,3.674276,0.158870,2.018282,3.028129,-0.703233,0.055686,-0.094391,-0.222053,0.028834,-0.025255,...,0.059988,0.165187,0.107744,1.252709,17.788628,26.339865,-70.781761,0.100169,0.088349,2021-05-28 04:00:00+00:00
5469,3.577648,0.159137,1.922177,2.911059,-0.826121,0.057259,-0.200336,-0.327967,-0.128102,-0.026969,...,0.060041,0.132124,0.073648,-1.098397,17.861473,24.262377,-70.906380,0.100170,0.087859,2021-05-29 00:00:00+00:00
5684,3.582210,0.159068,1.926682,2.914107,-0.825210,0.057296,-0.209619,-0.327081,-0.154174,-0.027088,...,0.059508,0.131564,0.065785,-0.876161,17.693375,24.584658,-71.180664,0.100258,0.087721,2021-05-31 05:45:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52061,4.338037,0.114227,3.175937,4.475598,0.282626,0.038629,1.067615,0.881781,1.125024,0.006486,...,0.038456,0.213316,0.283118,-37.429581,-2.051684,21.726267,-28.703650,0.101173,0.101710,2022-09-26 08:00:00+00:00
52068,4.255479,0.112811,3.102212,4.374035,0.179814,0.037806,0.965677,0.781383,0.995967,0.003754,...,0.038227,0.183495,0.250049,-37.723492,-1.839628,19.898615,-28.691217,0.101034,0.101248,2022-09-26 09:45:00+00:00
52099,4.189775,0.111435,3.037528,4.290281,0.098884,0.037965,0.885069,0.691254,0.886836,0.001455,...,0.038717,0.156247,0.222395,-37.991940,-1.294347,18.403667,-25.630245,0.101144,0.101195,2022-09-26 17:30:00+00:00
52164,4.068683,0.113271,2.920768,4.121860,-0.073968,0.040278,0.728442,0.533596,0.665782,-0.001619,...,0.040842,0.106598,0.168945,-36.815289,0.698980,15.664148,-24.706783,0.101926,0.101411,2022-09-27 09:45:00+00:00


In [ ]:
### save it on google drive
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/MasterProject/'+'create_data_my_model_add_label_2_lr_0.1'+'.parquet'
df_combined.to_parquet(file_path, index=False)

Mounted at /content/drive


In [ ]:
#0.8765328998395648
#0.8723729179726375    result of change hidden layer from 8 to 32
#0.9065247187071686    result of add one layer 32 between encoder :)
#0.904416749009233     result add hidden layer 32 to decoder too and add decoder to ruad model with gpu (this is pretty good)
#0.8655186444515994    result first ruad model with gpu and add timestamp column and woindow 10 and seq , adam optimizer
#0.8967381497429511    result first ruad model with gpu and add timestamp column and woindow 10 and remove seq , adam optimizer
#0.3060959342747257    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, adam optimizer
#0.34766214604908874   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer
#0.36209899126422795   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 +3 (i forgot to reset)
#0.907213292243243     result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 , add , h,c
#0.9098181020482538    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 6 , add , h,c
#0.9069957536027086    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq, SGD optimizer , epoch 6 , add , h,c